In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

import numpy as np
import pandas as pd

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

import warnings
from tqdm import tqdm
import os
from pathlib import Path
# Define the device
device = "cuda" if torch.cuda.is_available() else "mps" if  torch.backends.mps.is_available() else "cpu" # torch.has_mps or
print("Using device:", device)
if (device == 'cuda'):
    print(f"Device name: {torch.cuda.get_device_name(device.index)}")
    print(f"Device memory: {torch.cuda.get_device_properties(device.index).total_memory / 1024 ** 3} GB")
elif (device == 'mps'):
    print(f"Device name: <mps>")
else:
    print("NOTE: If you have a GPU, consider using it for training.")
    print("      On a Windows machine with NVidia GPU, check this video: https://www.youtube.com/watch?v=GMSjDTU8Zlc")
    print("      On a Mac machine, run: pip3 install --pre torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/nightly/cpu")
device = torch.device(device)

Using device: cuda
Device name: Tesla T4
Device memory: 14.74810791015625 GB


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        refined_enzy_embed, refined_smiles_embed = model(ESP_val_df_enzy,ESP_val_df_smiles)
        cos_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
        loss = loss_fn(cos_sim, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (cos_sim > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(cos_sim.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


### esm2 480 + MolFormer
Contrastive learning strategy

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_480')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(480, 480) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(480, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(480)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_480_MolFormer.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_480_MolFormer.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_480_MolFormer.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_480_MolFormer.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_480_MolFormer.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_480_MolFormer.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_480_MolFormer.pt')
    y_val = torch.load('ESP_val_df_label_esm2_480_MolFormer.pt')
    y_test = torch.load('ESP_test_df_label_esm2_480_MolFormer.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([50077, 480]) torch.Size([5422, 480]) torch.Size([13336, 480])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_1.pt")

Epoch 0: 100%|██████████| 3130/3130 [00:24<00:00, 127.15batch/s, train_loss=tensor(0.2326, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.1473784355671972 ACC=  0.799151604573958 bacc=  0.6504259846709604 precision=  0.7766990291262136 specificity=  0.9654221999498872 sensitivity=  0.33542976939203356 recall=  0.33542976939203356 MCC=  0.41727061558130796 AUC=  0.8252109349460464 f1=  0.4685212298682284
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1474); Accuracy (79.92)


Epoch 1: 100%|██████████| 3130/3130 [00:23<00:00, 131.55batch/s, train_loss=tensor(0.2078, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.13942457492680324 ACC=  0.816857248247879 bacc=  0.6770211137183051 precision=  0.8358895705521472 specificity=  0.9731896767727387 sensitivity=  0.3808525506638714 recall=  0.3808525506638714 MCC=  0.47976963516558246 AUC=  0.8538853405487296 f1=  0.5232837253960634
Epoch: 1 / 500, ############## the best accuracy in val  79.9152 at Epoch: 0  ##############
Performance in Val: Loss: (0.1394); Accuracy (81.69)


Epoch 2: 100%|██████████| 3130/3130 [00:24<00:00, 126.99batch/s, train_loss=tensor(0.1807, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.06523920915084053 ACC=  0.9302840280339358 bacc=  0.8961641681204093 precision=  0.903448275862069 specificity=  0.9684289651716362 sensitivity=  0.8238993710691824 recall=  0.8238993710691824 MCC=  0.8169024139902422 AUC=  0.9524376212655974 f1=  0.861842105263158
Epoch: 497 / 500, ############## the best accuracy in val  93.2128 at Epoch: 479  ##############
Performance in Val: Loss: (0.0652); Accuracy (93.03)


Epoch 498: 100%|██████████| 3130/3130 [00:23<00:00, 131.64batch/s, train_loss=tensor(0.0182, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.06644920991791363 ACC=  0.9297307266691258 bacc=  0.8971330672209537 precision=  0.8977272727272727 specificity=  0.9661738912553245 sensitivity=  0.8280922431865828 recall=  0.8280922431865828 MCC=  0.8157215444653666 AUC=  0.9519200346131697 f1=  0.8615049073064339
Epoch: 498 / 500, ############## the best accuracy in val  93.2128 at Epoch: 479  ##############
Performance in Val: Loss: (0.0664); Accuracy (92.97)


Epoch 499: 100%|██████████| 3130/3130 [00:23<00:00, 132.50batch/s, train_loss=tensor(0.0247, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.06597465718752954 ACC=  0.9306528956104758 bacc=  0.8991042213954143 precision=  0.8975131876412962 specificity=  0.9659233274868454 sensitivity=  0.8322851153039832 recall=  0.8322851153039832 MCC=  0.8183038290848086 AUC=  0.9517389843430037 f1=  0.8636693255982597
Epoch: 499 / 500, ############## the best accuracy in val  93.2128 at Epoch: 479  ##############
Performance in Val: Loss: (0.0660); Accuracy (93.07)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_1.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_480_MolFormer.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.0718527814056233 ACC=  0.9220905818836233 bacc=  0.8830117889936955 precision=  0.8930498254522374 specificity=  0.9656822810590632 sensitivity=  0.8003412969283277 recall=  0.8003412969283277 MCC=  0.7945433481651466 AUC=  0.9467973146905969 f1=  0.8441577921103945


(0.0718527814056233, 0.9220905818836233, 0.8830117889936955)

### esm2 1280 + MolFormer
Contrastive learning strategy

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_1280')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(1280, 1280) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(1280, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(1280)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_1280.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_1280.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_1280.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_1280.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_1280.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_1280.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_1280.pt')
    y_val = torch.load('ESP_val_df_label_esm2_1280.pt')
    y_test = torch.load('ESP_test_df_label_esm2_1280.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([50077, 1280]) torch.Size([5422, 1280]) torch.Size([13336, 1280])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 300):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_esm2_1280_second.pt")

In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_esm2_1280_second.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_1280.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.06455622224318092 ACC=  0.9293641271745651 bacc=  0.8966232920672214 precision=  0.8967324290998767 specificity=  0.9658859470468432 sensitivity=  0.8273606370875995 recall=  0.8273606370875995 MCC=  0.814628019995566 AUC=  0.9557602111036194 f1=  0.8606508875739646


(0.06455622224318092, 0.9293641271745651, 0.8966232920672214)

### esm2 2560 + MolFormer
Contrastive learning strategy


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        self.enzy_refine_layer_1 = nn.Linear(2560, 2560) # W1 and b
        self.smiles_refine_layer_1 = nn.Linear(768, 768) # W1 and b
        self.enzy_refine_layer_2 = nn.Linear(2560, 128) # W1 and b
        self.smiles_refine_layer_2 = nn.Linear(768, 128) # W1 and b

        self.relu = nn.ReLU()
        self.batch_norm_enzy = nn.BatchNorm1d(2560)
        self.batch_norm_smiles = nn.BatchNorm1d(768)
        self.batch_norm_shared = nn.BatchNorm1d(128)

    def forward(self, enzy_embed, smiles_embed):
        refined_enzy_embed = self.enzy_refine_layer_1(enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_1(smiles_embed)

        refined_enzy_embed = self.batch_norm_enzy(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_smiles(refined_smiles_embed)

        refined_enzy_embed = self.relu(refined_enzy_embed)
        refined_smiles_embed = self.relu(refined_smiles_embed)

        refined_enzy_embed = self.enzy_refine_layer_2(refined_enzy_embed)
        refined_smiles_embed = self.smiles_refine_layer_2(refined_smiles_embed)

        refined_enzy_embed = self.batch_norm_shared(refined_enzy_embed)
        refined_smiles_embed = self.batch_norm_shared(refined_smiles_embed)
        refined_enzy_embed = torch.nn.functional.normalize(refined_enzy_embed, dim=1)
        refined_smiles_embed = torch.nn.functional.normalize(refined_smiles_embed, dim=1)

        return refined_enzy_embed, refined_smiles_embed


In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)
    ESP_train_df_enzy_add = torch.load('ESP_train_df_enzy_>2800_<5500.pt')
    ESP_train_df_smiles_add = torch.load('ESP_train_df_smiles_>2800_<5500.pt')
    y_train_add = torch.load('ESP_train_df_label_>2800_<5500.pt')

    ESP_val_df_enzy_add = torch.load('ESP_val_df_enzy_>2800_<8000.pt')
    ESP_test_df_enzy_add = torch.load('ESP_test_df_enzy_>2800_<8000.pt')
    ESP_val_df_smiles_add = torch.load('ESP_val_df_smiles_>2800_<8000.pt')
    ESP_test_df_smiles_add = torch.load('ESP_test_df_smiles_>2800_<8000.pt')
    y_val_add = torch.load('ESP_val_df_label_>2800_<8000.pt')
    y_test_add = torch.load('ESP_test_df_label_>2800_<8000.pt')
    print(ESP_val_df_enzy_add.shape, ESP_val_df_smiles_add.shape, y_val_add.shape)
    print(ESP_test_df_enzy_add.shape, ESP_test_df_smiles_add.shape, y_test_add.shape)

    ESP_train_df_enzy = torch.cat([ESP_train_df_enzy, ESP_train_df_enzy_add], dim=0)
    ESP_train_df_smiles = torch.cat([ESP_train_df_smiles, ESP_train_df_smiles_add], dim=0)
    y_train = torch.cat([y_train, y_train_add], dim=0)
    print(ESP_train_df_enzy.shape, ESP_train_df_smiles.shape, y_train.shape)

    ESP_val_df_enzy = torch.cat([ESP_val_df_enzy, ESP_val_df_enzy_add], dim=0)
    ESP_val_df_smiles = torch.cat([ESP_val_df_smiles, ESP_val_df_smiles_add], dim=0)
    y_val = torch.cat([y_val, y_val_add], dim=0)
    print(ESP_val_df_enzy.shape, ESP_val_df_smiles.shape, y_val.shape)
    ESP_test_df_enzy = torch.cat([ESP_test_df_enzy, ESP_test_df_enzy_add], dim=0)
    ESP_test_df_smiles = torch.cat([ESP_test_df_smiles, ESP_test_df_smiles_add], dim=0)
    y_test = torch.cat([y_test, y_test_add], dim=0)
    print(ESP_test_df_enzy.shape, ESP_test_df_smiles.shape, y_test.shape)

    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.MSELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
torch.Size([23, 2560]) torch.Size([23, 768]) torch.Size([23, 1])
torch.Size([24, 2560]) torch.Size([24, 768]) torch.Size([24, 1])
torch.Size([50061, 2560]) torch.Size([50061, 768]) torch.Size([50061, 1])
torch.Size([5422, 2560]) torch.Size([5422, 768]) torch.Size([5422, 1])
torch.Size([13336, 2560]) torch.Size([13336, 768]) torch.Size([13336, 1])
3129


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
loss_train = []
loss_valid = []
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()

    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        num_batch = len(train_loader)
        loss_sum = 0
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            refined_enzy_embed, refined_smiles_embed = model(ESP_train_df_enzy,ESP_train_df_smiles)
            cosine_sim = torch.nn.functional.cosine_similarity(refined_enzy_embed, refined_smiles_embed, dim=1)
            loss = loss_fn(cosine_sim, y_train)
            loss_sum = loss_sum + loss.detach().cpu().numpy()
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum = loss_sum/num_batch
        loss_train.append(loss_sum)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        loss_valid.append(loss_sum_val)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_11.pt")

Epoch 0: 100%|██████████| 3129/3129 [00:29<00:00, 105.70batch/s, train_loss=tensor(0.2574, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.15453473258774542 ACC=  0.7897454813721874 bacc=  0.6328304022975525 precision=  0.7557117750439367 specificity=  0.9651716361814082 sensitivity=  0.3004891684136967 recall=  0.3004891684136967 MCC=  0.38205587781106 AUC=  0.8128040887244378 f1=  0.43
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.1545); Accuracy (78.97)


Epoch 1: 100%|██████████| 3129/3129 [00:29<00:00, 104.75batch/s, train_loss=tensor(0.1404, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.12694340112968006 ACC=  0.8284765769088898 bacc=  0.7055332919754282 precision=  0.8240620957309185 specificity=  0.9659233274868454 sensitivity=  0.4451432564640112 recall=  0.4451432564640112 MCC=  0.5182070709686368 AUC=  0.8796447317435578 f1=  0.5780399274047188
Epoch: 1 / 500, ############## the best accuracy in val  78.9745 at Epoch: 0  ##############
Performance in Val: Loss: (0.1269); Accuracy (82.85)


Epoch 2: 100%|██████████| 3129/3129 [00:29<00:00, 104.81batch/s, train_loss=tensor(0.1004, device='cuda:0', grad_fn=<MseLossBackward0>)]


...

loss_sum=  0.055989245178073556 ACC=  0.9402434526005164 bacc=  0.9127908513932729 precision=  0.9133681852128454 specificity=  0.970934602856427 sensitivity=  0.8546470999301188 recall=  0.8546470999301188 MCC=  0.8438024101208552 AUC=  0.9629408131958682 f1=  0.8830324909747292
Epoch: 497 / 500, ############## the best accuracy in val  94.2641 at Epoch: 483  ##############
Performance in Val: Loss: (0.0560); Accuracy (94.02)


Epoch 498: 100%|██████████| 3129/3129 [00:30<00:00, 102.94batch/s, train_loss=tensor(0.0146, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.055180798875371265 ACC=  0.9422722242714865 bacc=  0.915065448622083 precision=  0.9184131736526946 specificity=  0.9726885492357805 sensitivity=  0.8574423480083857 recall=  0.8574423480083857 MCC=  0.8490921954817683 AUC=  0.9646719969687212 f1=  0.8868810986628117
Epoch: 498 / 500, ############## the best accuracy in val  94.2641 at Epoch: 483  ##############
Performance in Val: Loss: (0.0552); Accuracy (94.23)


Epoch 499: 100%|██████████| 3129/3129 [00:30<00:00, 101.22batch/s, train_loss=tensor(0.0125, device='cuda:0', grad_fn=<MseLossBackward0>)]


loss_sum=  0.05603941356774233 ACC=  0.9406123201770564 bacc=  0.9150585322916465 precision=  0.9092250922509225 specificity=  0.9691806564770734 sensitivity=  0.8609364081062194 recall=  0.8609364081062194 MCC=  0.8450711755279224 AUC=  0.9636193139665576 f1=  0.8844221105527639
Epoch: 499 / 500, ############## the best accuracy in val  94.2641 at Epoch: 483  ##############
Performance in Val: Loss: (0.0560); Accuracy (94.06)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_11.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_add_>2800_<8000_in_valid2.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.06158351360281988 ACC=  0.9356628674265147 bacc=  0.9051909339672698 precision=  0.9084204056545789 specificity=  0.969653767820774 sensitivity=  0.8407281001137656 recall=  0.8407281001137656 MCC=  0.8313574088968375 AUC=  0.9593826968481587 f1=  0.8732644017725258


(0.06158351360281988, 0.9356628674265147, 0.9051909339672698)

### esm2 480 + MolFormer
simple concatenation

In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_480')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(1248, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)

        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_480_MolFormer.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_480_MolFormer.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_480_MolFormer.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_480_MolFormer.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_480_MolFormer.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_480_MolFormer.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_480_MolFormer.pt')
    y_val = torch.load('ESP_val_df_label_esm2_480_MolFormer.pt')
    y_test = torch.load('ESP_test_df_label_esm2_480_MolFormer.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([50077, 480]) torch.Size([5422, 480]) torch.Size([13336, 480])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_simple.pt")

Epoch 0: 100%|██████████| 3130/3130 [00:18<00:00, 172.90batch/s, train_loss=tensor(0.7019, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5615437252331624 ACC=  0.7375507192917743 bacc=  0.5088465994679503 precision=  0.5645161290322581 specificity=  0.9932347782510649 sensitivity=  0.02445842068483578 recall=  0.02445842068483578 MCC=  0.07334808874317308 AUC=  0.6234953698231923 f1=  0.04688546550569323
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5615); Accuracy (73.76)


Epoch 1: 100%|██████████| 3130/3130 [00:18<00:00, 171.83batch/s, train_loss=tensor(0.5061, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5352895368868622 ACC=  0.7393950571744744 bacc=  0.5067375564271883 precision=  0.9090909090909091 specificity=  0.9994988724630418 sensitivity=  0.013976240391334731 recall=  0.013976240391334731 MCC=  0.09342968218860954 AUC=  0.6692523236681556 f1=  0.02752924982794219
Epoch: 1 / 500, ############## the best accuracy in val  73.7551 at Epoch: 0  ##############
Performance in Val: Loss: (0.5353); Accuracy (73.94)


Epoch 2: 100%|██████████| 3130/3130 [00:18<00:00, 172.65batch/s, train_loss=tensor(0.7966, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.38751365047959185 ACC=  0.8690520103282922 bacc=  0.8586048168126713 precision=  0.7154811715481172 specificity=  0.8807316462039589 sensitivity=  0.8364779874213837 recall=  0.8364779874213837 MCC=  0.6843859804393891 AUC=  0.9171284411589248 f1=  0.7712628865979383
Epoch: 497 / 500, ############## the best accuracy in val  88.2516 at Epoch: 358  ##############
Performance in Val: Loss: (0.3875); Accuracy (86.91)


Epoch 498: 100%|██████████| 3130/3130 [00:18<00:00, 173.40batch/s, train_loss=tensor(0.2473, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.39138756432552657 ACC=  0.8758760604942826 bacc=  0.8471033094903786 precision=  0.7540214477211796 specificity=  0.9080430969681784 sensitivity=  0.7861635220125787 recall=  0.7861635220125787 MCC=  0.6851233106164298 AUC=  0.91959398163688 f1=  0.7697570988710231
Epoch: 498 / 500, ############## the best accuracy in val  88.2516 at Epoch: 358  ##############
Performance in Val: Loss: (0.3914); Accuracy (87.59)


Epoch 499: 100%|██████████| 3130/3130 [00:18<00:00, 170.82batch/s, train_loss=tensor(0.1393, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.3989415042549454 ACC=  0.8775359645887126 bacc=  0.8379211366735182 precision=  0.7757009345794392 specificity=  0.9218241042345277 sensitivity=  0.7540181691125087 recall=  0.7540181691125087 MCC=  0.6820811848647543 AUC=  0.9216448049340926 f1=  0.7647058823529412
Epoch: 499 / 500, ############## the best accuracy in val  88.2516 at Epoch: 358  ##############
Performance in Val: Loss: (0.3989); Accuracy (87.75)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_simple.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_480_simple_new.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.42307043597849653 ACC=  0.8642021595680864 bacc=  0.8125741735771764 precision=  0.7630360999691453 specificity=  0.9217922606924643 sensitivity=  0.7033560864618885 recall=  0.7033560864618885 MCC=  0.642200422886368 AUC=  0.9077927293095979 f1=  0.7319816486606482


(0.42307043597849653, 0.8642021595680864, 0.8125741735771764)

### esm2 1280 + MolFormer
simple concatenation


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_1280')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(2048, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)
        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_1280.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_1280.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_1280.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_1280.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_1280.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_1280.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_1280.pt')
    y_val = torch.load('ESP_val_df_label_esm2_1280.pt')
    y_test = torch.load('ESP_test_df_label_esm2_1280.pt')
    print(y_train.shape,y_val.shape, y_test.shape)


    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([50077, 1280]) torch.Size([5422, 1280]) torch.Size([13336, 1280])
torch.Size([50077, 768]) torch.Size([5422, 768]) torch.Size([13336, 768])
torch.Size([50077, 1]) torch.Size([5422, 1]) torch.Size([13336, 1])
3130


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model.pt")

Epoch 0: 100%|██████████| 3130/3130 [00:18<00:00, 168.70batch/s, train_loss=tensor(0.5457, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5702747650730223 ACC=  0.7366285503504242 bacc=  0.5113579278043663 precision=  0.5157894736842106 specificity=  0.9884740666499624 sensitivity=  0.03424178895877009 recall=  0.03424178895877009 MCC=  0.07631090941982721 AUC=  0.5974840140841001 f1=  0.06422018348623854
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5703); Accuracy (73.66)


Epoch 1: 100%|██████████| 3130/3130 [00:17<00:00, 174.18batch/s, train_loss=tensor(0.6738, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5422627621256144 ACC=  0.7401327923275544 bacc=  0.5088075528429533 precision=  0.84375 specificity=  0.9987471811576046 sensitivity=  0.018867924528301886 recall=  0.018867924528301886 MCC=  0.10136232466566568 AUC=  0.6545156371227296 f1=  0.03691045796308954
Epoch: 1 / 500, ############## the best accuracy in val  73.6629 at Epoch: 0  ##############
Performance in Val: Loss: (0.5423); Accuracy (74.01)


Epoch 2: 100%|██████████| 3130/3130 [00:18<00:00, 169.20batch/s, train_loss=tensor(0.4947, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.5114526716007413 ACC=  0.8959793434157137 bacc=  0.8502252009719282 precision=  0.8363072148952676 specificity=  0.9471310448509146 sensitivity=  0.753319357092942 recall=  0.753319357092942 MCC=  0.725235255856677 AUC=  0.9345380705469206 f1=  0.7926470588235294
Epoch: 497 / 500, ############## the best accuracy in val  89.8930 at Epoch: 485  ##############
Performance in Val: Loss: (0.5115); Accuracy (89.60)


Epoch 498: 100%|██████████| 3130/3130 [00:19<00:00, 163.31batch/s, train_loss=tensor(0.2611, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.4587721944860269 ACC=  0.8933972703799336 bacc=  0.8628151986273798 precision=  0.7980433263452131 specificity=  0.9275870709095465 sensitivity=  0.7980433263452131 recall=  0.7980433263452131 MCC=  0.7256303972547596 AUC=  0.9309250320558785 f1=  0.7980433263452131
Epoch: 498 / 500, ############## the best accuracy in val  89.8930 at Epoch: 485  ##############
Performance in Val: Loss: (0.4588); Accuracy (89.34)


Epoch 499: 100%|██████████| 3130/3130 [00:18<00:00, 171.66batch/s, train_loss=tensor(0.2083, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.4527898088426568 ACC=  0.8921062338620436 bacc=  0.8565592464246512 precision=  0.8043165467625899 specificity=  0.9318466549736908 sensitivity=  0.7812718378756115 recall=  0.7812718378756115 MCC=  0.7198710653709991 AUC=  0.9338001243538703 f1=  0.7926267281105991
Epoch: 499 / 500, ############## the best accuracy in val  89.8930 at Epoch: 485  ##############
Performance in Val: Loss: (0.4528); Accuracy (89.21)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_1280_simple_new.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.5352694592072978 ACC=  0.8758998200359928 bacc=  0.8321110477792529 precision=  0.7786762503743636 specificity=  0.924745417515275 sensitivity=  0.7394766780432309 recall=  0.7394766780432309 MCC=  0.675539020731937 AUC=  0.9187849869899372 f1=  0.7585703865791393


(0.5352694592072978, 0.8758998200359928, 0.8321110477792529)

In [ ]:
run_validation(model_test,val_loader,loss_fn, device) # performance evaluation

loss_sum=  0.46758457411281085 ACC=  0.898930284028034 bacc=  0.86388416564804 precision=  0.8206245461147422 specificity=  0.9381107491856677 sensitivity=  0.7896575821104123 recall=  0.7896575821104123 MCC=  0.7369323150478793 AUC=  0.9346475061550963 f1=  0.8048433048433048


(0.46758457411281085, 0.898930284028034, 0.86388416564804)

### esm2 2560 + MolFormer
simple concatenation

In [ ]:
def run_validation(model, val_loader,loss_fn, device):
    model.eval()
    loss_sum = 0
    num_batch = len(val_loader)
    total_y_true=[]
    total_y_pred=[]
    total_y_prob=[]
    for ESP_val_df_enzy,ESP_val_df_smiles, y_val in val_loader:

        ESP_val_df_enzy = ESP_val_df_enzy.to(device)
        ESP_val_df_smiles = ESP_val_df_smiles.to(device)
        y_val = y_val.squeeze(1).to(device)

        output = model(ESP_val_df_enzy,ESP_val_df_smiles)

        loss = loss_fn(output, y_val).detach().cpu().numpy()
        loss_sum = loss_sum + loss # count all the loss in the training process
        y_pred = (output > 0.5).float().cpu().numpy() # if score > 0.5, assign label 1 otherwise 0, transfer to cpu as numpy
        total_y_true.append(y_val.cpu().numpy())
        total_y_pred.append(y_pred)
        total_y_prob.append(output.detach().cpu().numpy())

    loss_sum = loss_sum/num_batch # get the overall average loss (Notice: this method is not 100% accurate)

    arrange_y_true = np.concatenate(total_y_true, axis=0)
    arrange_y_pred = np.concatenate(total_y_pred, axis=0)
    arrange_y_prob = np.concatenate(total_y_prob, axis=0)
    tn,fp,fn,tp = confusion_matrix(arrange_y_true, arrange_y_pred).ravel()
    acc = (tp+tn)/(tp+tn+fp+fn)
    specificity = tn/(tn+fp)
    sensitivity = tp/(tp+fn)
    recall = tp/(tp+fn)
    precision = tp/(tp+fp)
    bacc = (sensitivity + specificity)/2
    MCC = (tp*tn-fp*fn)/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
    AUC = roc_auc_score(arrange_y_true, arrange_y_prob)
    f1 = 2*precision*recall/(precision+recall)
    print("loss_sum= ",loss_sum, "ACC= ",acc, "bacc= ",bacc, "precision= ",precision,"specificity= ",specificity, "sensitivity= ",sensitivity, "recall= ",recall, "MCC= ",MCC, "AUC= ",AUC, "f1= ",f1)
    return loss_sum, acc, bacc   # , precision, sensitivity, recall, MCC, AUC, f1


In [ ]:
import os
os.chdir('/content/drive/MyDrive/EC_number_kroll/esm2_2560')

In [ ]:
import torch
import torch.nn as nn

class Contrastive_learning_layer(nn.Module):
    def __init__(self):
        super().__init__()
        # self.combined_embed_layer_1 = nn.Linear(2048, 2048)
        self.combined_embed_layer_2 = nn.Linear(3328, 128)
        self.combined_embed_layer_3 = nn.Linear(128, 1)
        # self.batch_norm_1= nn.BatchNorm1d(2048)
        # self.batch_norm_2= nn.BatchNorm1d(128)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.15)
        # self.relu_1 = nn.ReLU()
        self.relu_2 = nn.ReLU()

    def forward(self, enzy_embed, smiles_embed):
        combined_embed = torch.cat([enzy_embed, smiles_embed], dim=1)
        # combined_embed = self.combined_embed_layer_1(combined_embed)
        # combined_embed = self.batch_norm_1(combined_embed)

        # combined_embed = self.relu_1(combined_embed)
        combined_embed = self.combined_embed_layer_2(combined_embed)
        # combined_embed = self.batch_norm_2(combined_embed)
        combined_embed = self.relu_2(combined_embed)
        combined_embed = self.dropout(combined_embed)
        combined_embed = self.combined_embed_layer_3(combined_embed)
        # combined_embed = torch.nn.functional.normalize(combined_embed, dim=1)
        return self.sigmoid(combined_embed).squeeze(1)

In [ ]:
def get_ds():
    # Load the saved embeddings_results
    ESP_train_df_enzy = torch.load('ESP_train_df_enzy_esm2_2560.pt')
    ESP_val_df_enzy = torch.load('ESP_val_df_enzy_esm2_2560.pt')
    ESP_test_df_enzy = torch.load('ESP_test_df_enzy_esm2_2560.pt')
    print(ESP_train_df_enzy.shape, ESP_val_df_enzy.shape, ESP_test_df_enzy.shape)
    # Load the saved embeddings_results
    ESP_train_df_smiles = torch.load('ESP_train_df_smiles_esm2_2560.pt')
    ESP_val_df_smiles = torch.load('ESP_val_df_smiles_esm2_2560.pt')
    ESP_test_df_smiles = torch.load('ESP_test_df_smiles_esm2_2560.pt')
    print(ESP_train_df_smiles.shape, ESP_val_df_smiles.shape, ESP_test_df_smiles.shape)

    y_train = torch.load( 'ESP_train_df_label_esm2_2560.pt')
    y_val = torch.load('ESP_val_df_label_esm2_2560.pt')
    y_test = torch.load('ESP_test_df_label_esm2_2560.pt')
    print(y_train.shape,y_val.shape, y_test.shape)
    ESP_train_df_enzy_add = torch.load('ESP_train_df_enzy_>2800_<5500.pt')
    ESP_train_df_smiles_add = torch.load('ESP_train_df_smiles_>2800_<5500.pt')
    y_train_add = torch.load('ESP_train_df_label_>2800_<5500.pt')

    ESP_val_df_enzy_add = torch.load('ESP_val_df_enzy_>2800_<8000.pt')
    ESP_test_df_enzy_add = torch.load('ESP_test_df_enzy_>2800_<8000.pt')
    ESP_val_df_smiles_add = torch.load('ESP_val_df_smiles_>2800_<8000.pt')
    ESP_test_df_smiles_add = torch.load('ESP_test_df_smiles_>2800_<8000.pt')
    y_val_add = torch.load('ESP_val_df_label_>2800_<8000.pt')
    y_test_add = torch.load('ESP_test_df_label_>2800_<8000.pt')
    print(ESP_val_df_enzy_add.shape, ESP_val_df_smiles_add.shape, y_val_add.shape)
    print(ESP_test_df_enzy_add.shape, ESP_test_df_smiles_add.shape, y_test_add.shape)

    ESP_train_df_enzy = torch.cat([ESP_train_df_enzy, ESP_train_df_enzy_add], dim=0)
    ESP_train_df_smiles = torch.cat([ESP_train_df_smiles, ESP_train_df_smiles_add], dim=0)
    y_train = torch.cat([y_train, y_train_add], dim=0)
    print(ESP_train_df_enzy.shape, ESP_train_df_smiles.shape, y_train.shape)

    ESP_val_df_enzy = torch.cat([ESP_val_df_enzy, ESP_val_df_enzy_add], dim=0)
    ESP_val_df_smiles = torch.cat([ESP_val_df_smiles, ESP_val_df_smiles_add], dim=0)
    y_val = torch.cat([y_val, y_val_add], dim=0)
    print(ESP_val_df_enzy.shape, ESP_val_df_smiles.shape, y_val.shape)
    ESP_test_df_enzy = torch.cat([ESP_test_df_enzy, ESP_test_df_enzy_add], dim=0)
    ESP_test_df_smiles = torch.cat([ESP_test_df_smiles, ESP_test_df_smiles_add], dim=0)
    y_test = torch.cat([y_test, y_test_add], dim=0)
    print(ESP_test_df_enzy.shape, ESP_test_df_smiles.shape, y_test.shape)

    train_tensor_dataset = TensorDataset(ESP_train_df_enzy,ESP_train_df_smiles, y_train)
    val_tensor_dataset = TensorDataset(ESP_val_df_enzy,ESP_val_df_smiles, y_val)
    test_tensor_dataset = TensorDataset(ESP_test_df_enzy, ESP_test_df_smiles, y_test)

    # Create TensorDataset and DataLoaders
    batch_size = 16
    train_loader = DataLoader(train_tensor_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_tensor_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_tensor_dataset, batch_size=batch_size, shuffle=False)
    print(len(train_loader))
    return train_loader,  val_loader, test_loader

In [ ]:
# design the model, optimizer and loss function
model = Contrastive_learning_layer().to(device)
train_loader,  val_loader, test_loader = get_ds()
optimizer = torch.optim.Adam(model.parameters(),lr= 1e-03)
loss_fn = nn.BCELoss().to(device)

torch.Size([49881, 2560]) torch.Size([5399, 2560]) torch.Size([13312, 2560])
torch.Size([49881, 768]) torch.Size([5399, 768]) torch.Size([13312, 768])
torch.Size([49881, 1]) torch.Size([5399, 1]) torch.Size([13312, 1])
torch.Size([23, 2560]) torch.Size([23, 768]) torch.Size([23, 1])
torch.Size([24, 2560]) torch.Size([24, 768]) torch.Size([24, 1])
torch.Size([50061, 2560]) torch.Size([50061, 768]) torch.Size([50061, 1])
torch.Size([5422, 2560]) torch.Size([5422, 768]) torch.Size([5422, 1])
torch.Size([13336, 2560]) torch.Size([13336, 768]) torch.Size([13336, 1])
3129


In [ ]:
initial_epoch = 0
best_epoch = -1
best_accuracy = 0.5
for epoch in range(initial_epoch, 500):
    torch.cuda.empty_cache()
    model.train()
    with tqdm(train_loader, desc='Processing', unit="batch") as tepoch:
        for ESP_train_df_enzy,ESP_train_df_smiles, y_train in tepoch:
            model.train()
            tepoch.set_description(f"Epoch {epoch}")
            ESP_train_df_enzy = ESP_train_df_enzy.to(device)
            ESP_train_df_smiles = ESP_train_df_smiles.to(device)
            y_train = y_train.squeeze(1).to(device)
            output = model(ESP_train_df_enzy,ESP_train_df_smiles)

            loss = loss_fn(output, y_train)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad(set_to_none=True) #
            tepoch.set_postfix(train_loss=loss)
            # tepoch.set_postfix(val_loss=loss_sum_val, val_accuracy=100. * acc_val, balanced_val_accuracy=100. * bacc_val)
        loss_sum_val,acc_val, bacc_val = run_validation(model,val_loader,loss_fn, device)
        print('Epoch: %d / %d, ############## the best accuracy in val  %.4f at Epoch: %d  ##############'  % (epoch, 500,100 * best_accuracy,best_epoch))
        # print('Performance in Train: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum, 100 * acc))
        print('Performance in Val: Loss: (%.4f); Accuracy (%.2f)' % (loss_sum_val, 100 * acc_val))
        # checkpoint(model, f"epoch-{epoch}.pth")
        if acc_val > best_accuracy: # compare the performance updates at the val set
            best_accuracy = acc_val
            best_epoch = epoch
            torch.save(model, "best_model_simple_concatenate.pt")

Epoch 0: 100%|██████████| 3129/3129 [00:18<00:00, 166.64batch/s, train_loss=tensor(0.5717, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5667580816246416 ACC=  0.7375507192917743 bacc=  0.5039158687059861 precision=  0.7222222222222222 specificity=  0.9987471811576046 sensitivity=  0.009084556254367574 recall=  0.009084556254367574 MCC=  0.06001017258279577 AUC=  0.5834687270677683 f1=  0.017943409247757072
Epoch: 0 / 500, ############## the best accuracy in val  50.0000 at Epoch: -1  ##############
Performance in Val: Loss: (0.5668); Accuracy (73.76)


Epoch 1: 100%|██████████| 3129/3129 [00:17<00:00, 182.58batch/s, train_loss=tensor(0.5686, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.5446345146778411 ACC=  0.7399483585392844 bacc=  0.5080098985820822 precision=  0.8888888888888888 specificity=  0.9992483086945627 sensitivity=  0.016771488469601678 recall=  0.016771488469601678 MCC=  0.10030904301767554 AUC=  0.6324102571106442 f1=  0.03292181069958848
Epoch: 1 / 500, ############## the best accuracy in val  73.7551 at Epoch: 0  ##############
Performance in Val: Loss: (0.5446); Accuracy (73.99)


Epoch 2: 100%|██████████| 3129/3129 [00:17<00:00, 180.23batch/s, train_loss=tensor(0.7590, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


...

loss_sum=  0.5539681326405387 ACC=  0.9114717816303947 bacc=  0.8845060365556954 precision=  0.8355681016231475 specificity=  0.9416186419443748 sensitivity=  0.827393431167016 recall=  0.827393431167016 MCC=  0.7714497693010952 AUC=  0.9492700294740735 f1=  0.8314606741573033
Epoch: 497 / 500, ############## the best accuracy in val  91.7927 at Epoch: 445  ##############
Performance in Val: Loss: (0.5540); Accuracy (91.15)


Epoch 498: 100%|██████████| 3129/3129 [00:18<00:00, 172.71batch/s, train_loss=tensor(0.1113, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.555866645659221 ACC=  0.915529324972335 bacc=  0.8861416173812462 precision=  0.8512635379061372 specificity=  0.94838386369331 sensitivity=  0.8238993710691824 recall=  0.8238993710691824 MCC=  0.78051818567176 AUC=  0.9517380213096519 f1=  0.8373579545454546
Epoch: 498 / 500, ############## the best accuracy in val  91.7927 at Epoch: 445  ##############
Performance in Val: Loss: (0.5559); Accuracy (91.55)


Epoch 499: 100%|██████████| 3129/3129 [00:18<00:00, 172.85batch/s, train_loss=tensor(0.0113, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)]


loss_sum=  0.584132947481107 ACC=  0.9109184802655846 bacc=  0.8767340947600304 precision=  0.8500738552437223 specificity=  0.9491355549987471 sensitivity=  0.8043326345213138 recall=  0.8043326345213138 MCC=  0.7672303921771007 AUC=  0.9495398539095914 f1=  0.8265709156193897
Epoch: 499 / 500, ############## the best accuracy in val  91.7927 at Epoch: 445  ##############
Performance in Val: Loss: (0.5841); Accuracy (91.09)


In [ ]:
import torch
# Specify the file path where the entire model is saved
load_path = 'best_model_simple_concatenate.pt'
# Load the entire model
model_test = torch.load(load_path)
# model_test.to('cuda')
# rename the baseline model
torch.save(model_test, 'best_model_esm2_2560_simple_new_add_>2800_<8000_in_valid.pt')
run_validation(model_test,test_loader,loss_fn, device) # performance evaluation

loss_sum=  0.6975750319996458 ACC=  0.8964457108578284 bacc=  0.8533654124641732 precision=  0.8310077519379845 specificity=  0.9445010183299389 sensitivity=  0.7622298065984073 recall=  0.7622298065984073 MCC=  0.7272308542398608 AUC=  0.9356892639756805 f1=  0.7951342530781783


(0.6975750319996458, 0.8964457108578284, 0.8533654124641732)